# OpenADR 3 Demand Flexibility for Hot Water Heaters

## Overview

This project demonstrates how to use **OpenADR 3.0** to communicate demand flexibility signals for heat pump water heaters (HPWHs). The system fetches electricity pricing data, publishes it through an OpenADR 3 Virtual Top Node (VTN), and converts price signals into CTA-2045 control schedules for water heaters.

### What is OpenADR 3?

OpenADR (Open Automated Demand Response) is an open standard for communicating demand response signals between utilities/aggregators and end devices. Version 3.0 uses a REST API architecture with the following key concepts:

- **VTN (Virtual Top Node)** — the server that publishes programs, events, and price signals
- **VEN (Virtual End Node)** — the client that receives signals and controls devices
- **Programs** — define demand response programs with their parameters
- **Events** — time-based signals (e.g., price schedules) published under a program
- **Reports** — telemetry data sent from VENs back to the VTN
- **Subscriptions** — webhook registrations for change notifications

### Architecture

```
Pricing API                  OpenADR 3 VTN              VEN / Control Algorithm
(Olivine)                    (Reference Impl.)          (Thermal Systems)
┌──────────────┐             ┌──────────────┐           ┌──────────────────┐
│  Electricity │  fetch      │              │  OpenADR  │  Price → CTA2045 │
│  Price Data  │ ─────────> │  Programs &  │ ────────> │  Schedule for    │
│  (eTOU-Dyn)  │  & publish │  Events      │  signals  │  Water Heater    │
└──────────────┘             └──────────────┘           └──────────────────┘
```

---
## Prerequisites

| Requirement | macOS | Linux (Ubuntu/Debian) |
|---|---|---|
| **Python 3.13** | `brew install python@3.13` | `sudo add-apt-repository ppa:deadsnakes/ppa && sudo apt install python3.13 python3.13-venv` |
| **pip** | Included with Python from Homebrew | `sudo apt install python3-pip` |
| **virtualenv** | `pip install virtualenv` | `pip install virtualenv` |
| **Git** | `brew install git` (or Xcode CLT: `xcode-select --install`) | `sudo apt install git` |
| **curl** | Pre-installed on macOS | `sudo apt install curl` |

### Python packages for the quickstart notebook

```bash
pip install requests isodate matplotlib numpy jupyter
```

> **Note:** Docker is **not** required. All components run natively with Python.

---
## Component 1: OpenADR 3 VTN Reference Implementation

### What is it?

The VTN Reference Implementation is a fully functional OpenADR 3 server auto-generated from the OpenAPI specification using SwaggerHub, with added business logic. It serves as the central hub that publishes programs, events, and pricing signals that VENs consume.

> **For interested parties:** The business logic layer in the VTN can be modified to customize signals and price types. See the controller modules in `swagger_server/controllers/` to adapt the VTN for your specific use case.

### Setup

```bash
# Clone the repository
cd openadr3-vtn-reference-implementation

# Create and activate a virtual environment
virtualenv venv
source venv/bin/activate

# Install dependencies
pip3 install -r requirements.txt
```

### Dependencies

| Package | Version | Purpose |
|---|---|---|
| connexion | 2.6.0 | OpenAPI-based Flask framework |
| Flask | 2.2.5 | Web framework |
| jsonschema | 3.2.0 | JSON validation |
| PyJWT | 2.8.0 | JWT token handling |
| cryptography | 41.0.5 | Cryptographic operations |
| gevent | 25.9.1 | Async support |
| paho-mqtt | 1.6.1 | MQTT pub-sub messaging (optional) |
| requests | 2.25.1 | HTTP client |
| pytest | 8.4.2 | Testing |

### Running the VTN

```bash
python -m swagger_server
```

The VTN will start listening at:
- **Base URL:** `http://localhost:8080/openadr3/3.0.1`
- **API UI:** `http://localhost:8080/openadr3/3.0.1/ui/` (note: may not be working, see [Issue #81](https://github.com/oadr3-org/openadr3-vtn-reference-implementation/issues/81))

### Authentication

The VTN uses HTTP (not HTTPS) for local development. You must first obtain a token via the `/auth/token` endpoint using client credentials:

| Client ID | Client Secret | Role |
|---|---|---|
| `bl_client` | `1001` | Business Logic — create/update/delete programs, events, etc. |
| `ven_client` | `999` | VEN — read programs/events, submit reports |

```bash
# Get a Business Logic token
BL_TOKEN=$(curl -s -X POST http://localhost:8080/openadr3/3.0.1/auth/token \
  -H "Content-Type: application/x-www-form-urlencoded" \
  -d "grant_type=client_credentials&client_id=bl_client&client_secret=1001" \
  | python3 -c "import sys,json; print(json.load(sys.stdin)['access_token'])")

# Get a VEN token
VEN_TOKEN=$(curl -s -X POST http://localhost:8080/openadr3/3.0.1/auth/token \
  -H "Content-Type: application/x-www-form-urlencoded" \
  -d "grant_type=client_credentials&client_id=ven_client&client_secret=999" \
  | python3 -c "import sys,json; print(json.load(sys.stdin)['access_token'])")
```

### Verify it's running

```bash
curl -H "Authorization: Bearer $BL_TOKEN" \
     http://localhost:8080/openadr3/3.0.1/programs
```

Expected response: `[]` (empty list, no programs created yet)

### MQTT Messaging (Optional)

OpenADR 3.1.0+ supports optional pub-sub notifications via MQTT. To disable MQTT (default for this project), set `NOTIFIER_BINDINGS = []` in `config.py`. If you want MQTT support, install and run a [Mosquitto](https://mosquitto.org) broker and configure `config.py` accordingly.

---
## Component 2: OpenADR 3 Test Tool

### What is it?

The OpenADR Alliance Test Tool is a client application for testing and certifying both VTNs and VENs. It includes:

- **VTN tests** — comprehensive test scripts covering happy paths, authorization, error handling, and query parameters for all OpenADR 3 resources (programs, events, reports, subscriptions, VENs, resources)
- **VEN tests** — profiles for testing VEN interoperability (baseline, continuous pricing)
- **Web UI** — Django-based interface for configuring and running tests
- **Curl examples** — ready-to-use curl commands for every VTN API operation

### Setup

```bash
# Clone the repository
cd openadr3-test-tool

# Create and activate a virtual environment (use Python 3.9+)
virtualenv venv
source venv/bin/activate

# Install dependencies
pip install -r requirements.txt

# Use in-memory database (no Docker/PostgreSQL needed)
export IN_MEMORY_DATABASE=True

# Initialize the database
chmod +x init_database.sh
./init_database.sh
```

### Running the Test Tool UI

```bash
python manage.py runserver
```

Open your browser to: `http://127.0.0.1:8000/home`

Login with any username/password.

### Running Tests from Command Line

Make sure the VTN is running first (Component 1), then:

```bash
# Run all VTN tests
pytest ./VTN_test --html=./logs/pytest_report.html

# Run a specific test module
pytest ./VTN_test/test_subscriptions_api.py::TestSubscriptionsApi

# Run a specific test
pytest ./VTN_test/event_test/test_events.py::test_create_event_bl

# Run with custom VTN URL and tokens
pytest ./VTN_test \
    --vtn_base_url=http://localhost:8080/openadr3/3.0.1 \
    --ven_token=ven_token \
    --bl_token=bl_token
```

### Default Credentials & Configuration

| Variable | Default | Description |
|---|---|---|
| `VTN_BASE_URL` | `http://localhost:8080/openadr3/3.0.1` | VTN endpoint |
| `VEN_CLIENT_ID` | `ven_client` | VEN client ID |
| `VEN_CLIENT_SECRET` | `999` | VEN client secret |
| `BL_CLIENT_ID` | `bl_client` | Business logic client ID |
| `BL_CLIENT_SECRET` | `1001` | Business logic client secret |
| `TEST_CALLBACK_SERVICE_BASE_URL` | `http://localhost:5000` | Callback service for subscriptions |
| `TEST_CALLBACK_SERVICE_TOKEN` | `test-token` | Callback service auth token |
| `LOG_LEVEL` | `DEBUG` | Logging level (DEBUG/INFO/WARNING/ERROR) |

### Curl Examples

The `VTN_client_curl_examples/` folder contains ready-to-use curl commands for every API operation described in the OpenADR 3 User Guide. These are useful for manual testing and understanding the API.

---
## Component 3: Thermal Systems / Control Algorithms

### What is it?

The `thermal_systems` library (developed by LBNL) contains simulation models and control strategies for heat pump water heaters. The key module for this project is the **control algorithms** that convert electricity price signals into CTA-2045 set temperature schedules.

### Key Files

- **`controls/hpwh_fleet_control/control_algorithms.py`** — Price-responsive control algorithms including:
  - `MinMax_Supervisory` — determines set temperatures based on price thresholds (mean +/- standard deviation)
  - `StateOfCharge_Adjustment` — adjusts price thresholds based on tank state of charge

### How the Control Algorithm Works

1. Takes a price schedule (e.g., 24-hour ahead prices)
2. Computes mean and standard deviation of prices
3. **High price periods** (price > mean + stdev) → lower set temperature (48.9°C) to reduce consumption
4. **Low price periods** (price < mean - adjusted stdev) → higher set temperature to pre-heat
5. **Normal periods** → default set temperature (51.6°C)
6. State of charge adjustment makes pre-heating more likely when the tank is low

### Available Resources

The `thermal_systems/resources/` folder contains supporting data:

| Resource | Description |
|---|---|
| `electricity_prices/` | Price data for 25 climate zones |
| `emissions/` | Marginal CO2 emissions data |
| `weather/` | Weather datasets |
| `hot_water_draws/` | Hot water usage profiles |
| `hpwh_config_files/` | HPWH configuration files |
| `hpwh_signal_schedules/` | Control signal schedules |

### Dependencies

```bash
pip install pandas numpy matplotlib jupyter
```

---
## Component 4: Pricing Data Source

### API Endpoint

Electricity pricing data is sourced from the Olivine API:

```
https://api.olivineinc.com/i/oe/pricing/signal/paced/etou-dyn
```

This provides dynamic time-of-use (eTOU-Dyn) pricing signals with hourly intervals. Example response fields:

| Field | Description |
|---|---|
| `activePeriod.startTime` | Start time of the pricing window (ISO 8601) |
| `activePeriod.duration` | Duration of the full pricing window (e.g., `PT13H`) |
| `priceSignals[].interval` | Hour offset from start |
| `priceSignals[].priceUSD_per_kWh` | Price in USD per kWh |
| `signalMetadata.intervalDuration` | Duration of each interval (e.g., `PT1H`) |

### Sample Data Files

Pre-built sample data files are provided in `sample_data/`:

- **`program_pricing.json`** — OpenADR program definition for eTOU-Dyn pricing
- **`event_pricing.json`** — OpenADR event with 13 hourly price intervals from the Olivine API

> **Note:** The `event_pricing.json` file contains a `PROGRAM_ID_PLACEHOLDER` that must be replaced with the actual program ID returned when you create the program (see Quick Start Step 4).

### Fetching and Publishing Prices

The workflow is:
1. **Fetch** prices from the Olivine API (or use the sample data)
2. **Get a token** from the VTN (`POST /auth/token`)
3. **Create a program** on the VTN using `sample_data/program_pricing.json`
4. **Create an event** on the VTN using `sample_data/event_pricing.json` (with the real program ID)
5. VENs poll the VTN to get updated pricing events

---
## OpenADR 3.0.1 Specification Reference

The OpenADR 3.0.1 specification is included in the `OpenADR 3.0 Specification_3.0.1/` folder within this repository.

### Specification Files

| File | Description |
|---|---|
| `0_READ ME OpenADR 3.0 Information and Certification.pdf` | Overview of OpenADR 3.0 and the certification program |
| `1_oadr3.0.1.yaml` | **Normative reference** — OpenAPI 3.0 definition (source of truth for the API) |
| `2_OpenADR 3.0 Definition v3.0.1.pdf` | Information models, enumerations, security definitions |
| `3_OpenADR 3.0 User Guide v3.0.1.pdf` | Use case examples for programs, events, reports, and endpoints |

> **Note on the YAML spec:** The OpenAPI YAML is the normative reference and supersedes any statements in other documents. Report inconsistencies to comments@openadr.org.

### OpenADR 3.0 Components

1. **OpenAPI YAML Specification** — the normative API definition
2. **Definitions Document** — readable information models and enumerations
3. **User Guide** — common use scenarios (pricing, alerts, load control, etc.)
4. **Reference Implementation** — VTN server (available to OpenADR Alliance members)

### Key API Resources (3.0.1)

| Resource | Endpoint | Description |
|---|---|---|
| Programs | `/programs` | Define demand response programs |
| Events | `/events` | Time-based signals (prices, load control) under a program |
| Reports | `/reports` | Telemetry data from VENs |
| Subscriptions | `/subscriptions` | Webhook registrations for change notifications |
| VENs | `/vens` | Register and manage Virtual End Nodes |
| Resources | `/vens/{venID}/resources` | Devices/assets managed by a VEN |
| Auth | `/auth/token` | Obtain access tokens |

### Certification Profiles

OpenADR 3.0 defines certification profiles for different use cases:
- **Price Receiving, Emergency Alert VEN** — the profile most relevant to this project
- Demand Flexibility VEN (future)
- EVSE Management VEN (future)
- Inverter Management VEN (future)

### Important: 3.0.1 vs 3.1.0 Compatibility

The VTN reference implementation is internally based on the **3.1.0** specification but we have configured it to serve at the `/openadr3/3.0.1` base path. For the core operations used in this project (programs, events, reports, VENs), the two versions are compatible. Key differences to be aware of:

| Aspect | 3.0.1 (our target) | 3.1.0 (VTN internal) |
|---|---|---|
| Program fields | Has `bindingEvents`, `localPrice`, `retailerName`, `retailerLongName`, `programType`, `country`, `principalSubdivision` | Removed most of these; added `attributes` |
| Auth endpoint | Spec says GET with headers | Implementation uses POST with form-encoded body (we use the POST method) |
| Subscription operations | `GET`, `POST`, `PUT`, `DELETE` | `READ`, `CREATE`, `UPDATE`, `DELETE` |
| MQTT/Messaging | Not supported | Supported (optional, disabled in our setup) |
| Targets | Complex `valuesMap` arrays | Simplified `target` strings |

For the pricing/demand flexibility use case in this project, these differences do not affect functionality.

---
## Quick Start: End-to-End Setup

### Step 1: Start the VTN (in a separate terminal)

```bash
cd openadr3-vtn-reference-implementation
virtualenv venv && source venv/bin/activate
pip3 install -r requirements.txt
python -m swagger_server
```

### Step 2: Run the Quick Start notebook

Once the VTN is running, open and execute **[quickstart.ipynb](quickstart.ipynb)** which will:

1. **Connect to the VTN** and authenticate with client credentials
2. **Fetch live prices** from the Olivine API (`api.olivineinc.com`)
3. **Create a pricing program** on the VTN
4. **Publish price signals as an event** — converts Olivine data to OpenADR 3 event format
5. **Read events as a VEN** — demonstrates what a water heater controller would do
6. **Run control algorithm** (TODO) — convert prices to CTA-2045 set temperature schedules